# Vector boson fusion in IDM

The final notebook is [Significance_countours.ipynb](./Significance_countours.ipynb) which is based on [Sensitivity_fit.ipynb](Sensitivity_fit.ipynb) which is based on [Sensitivity_plots.ipynb](Sensitivity_plots.ipynb) by JD

### Initialization

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys
import subprocess
import numpy as np
import pandas as pd
#pip3 install pyslha
import pyslha

In [12]:
thisroot='/home/restrepo/prog/ROOT/root/bin/thisroot.sh'
thisroot='/opt/root5/bin/thisroot.sh'

INSTALL=True # If True check full installation
TEST=False #take a long time

## Install root 5

### Prerequisites 
```bash
apt-get install cmake git dpkg-dev make g++ gcc binutils libx11-dev libxpm-dev \
libxft-dev libxext-dev gfortran libssl-dev libpcre3-dev \
xlibmesa-glu-dev libglew1.5-dev libftgl-dev \
libmysqlclient-dev libfftw3-dev libcfitsio-dev \
graphviz-dev libavahi-compat-libdnssd-dev \
libldap2-dev python-dev libxml2-dev libkrb5-dev \
libgsl0-dev libqt4-dev r-base r-base-dev python-pip
```

Install ROOT 5 in some `PATH`
```bash
git clone http://root.cern.ch/git/root.git
cd root
git checkout v5-34-00-patches
./configure
make

```
Add to your `.bashrc`
```bash
source PATH/root/bin/thisroot.sh
```

In [23]:
f=open('kk.sh','w')
f.write('source '+thisroot+'\n')
f.write('which root\n')
f.close()

if not subprocess.Popen('bash kk.sh'.split(),
            stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()[0]:
    sys.exit('INSTALL ROOT: see instrucctions in notebook')

# Development of module to calculate one specific point

It is assumed that you did the clone as:
```bash
git clone  --recursive git@gitlab.com:restrepo/VBF_IDM.git
```

In [31]:
if INSTALL:
    s=subprocess.Popen('git checkout -b v2.3.3'.split(),cwd='madgraph',
                      stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()
    if 'Switched' not in s[1].decode('utf-8'):
        sys.exit('Submodule problems')

subprocess does not use .bashrc

In [125]:
f=open('madgraph/kk.sh','w')
f.write('source '+thisroot+'\n')
f.write('./bin/mg5_aMC install.dat\n')
f.close()

In [126]:
if INSTALL:
    s=subprocess.call('bash kk.sh'.split(),cwd='madgraph', stdout=open('kk','w'),stderr=open('kkk','w') )

In [ ]:
if TEST:
    s=subprocess.Popen('./test.sh'.split(), cwd='test',
            stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

s[0].decode('utf-8')

### Create MadGraph script with processes

In [4]:
LambdasL=[0.01,0.02,0.05,0.07,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,1.0,3.0,5.0,7.0,10.0]

In [6]:
MH0s=np.array([70,75,80,85,90,110,130,150,170,190,210,220,240])

In [7]:
len(LambdasL)

1

```bash
rsiii@gfif:~/IDM_VBF/Samples/mh0_vs_lal/scan$ ls delphes_events_[0-9][0-9]_*.root | more
delphes_events_70_001_.root
delphes_events_70_002_.root
delphes_events_70_003_.root
...
...
delphes_events_90_018_.root
delphes_events_90_019_.root
delphes_events_90_020_.root
```

```bash
rsiii@gfif:~/IDM_VBF/Samples/mh0_vs_lal/scan$ ls delphes_events_[0-9][0-9][0-9]_*.root | more
delphes_events_110_001_.root
delphes_events_110_002_.root
delphes_events_110_003_.root
...
...
delphes_events_240_018_.root
delphes_events_240_019_.root
delphes_events_240_020_.root
```

cat readme.txt 
date 23/01/2017
Delphes output (.root) for the set the points contained in data_mh0_vs_x.dat

The data is organized as follows (in ascending order in lambda_L[lal] ):

#lal #mh0[GeV] #xs[fb]   #Delphes name
0.3 63.75 200        --> delphes_events_2.root
0.4 64.80 200        --> delphes_events_3.root 
0.5 66.32 200        --> delphes_events_4.root
0.6 68.05 200        --> delphes_events_5.root 
0.7 70.04 200        --> delphes_events_6.root
.   .     .          --> 
.   .     .          --> 
.   .     .          -->

The cross-section (xs) for the processes p p > h0 h0 j j
for all the set of points is around 200 fb. [Except for
the first point, with lal=0.3,  where the cross-section turns out to be 179 fb.]

In [5]:
df=pd.read_csv('Output_data.csv')

In [6]:
lha=pyslha.readSLHAFile('MadGraph_cards/benchmarks/param_card_template.dat')

In [16]:
MHc=750
MH0=230
MADGRAPH='madgraph'
run_dir='Task_Asana'
work_dir='studies/IDM/'+run_dir
work_script='BP_'+str(int(MHc))+'_A_'+str(int(MH0))+'.txt'

LHA_input_file='MadGraph_cards/benchmarks/param_card_template.dat'#'MadGraph_cards/benchmarks/param_card_BP'''+str(int(MHc))+'_'+str(int(MH0))+'.dat'
#lha.blocks['FRBLOCK'].entries[13]=MH0
#lha.blocks['FRBLOCK'].entries[14]=MHc
#lha.blocks['FRBLOCK'].entries[15]=MHc
#pyslha.writeSLHAFile(LHA_input_file,lha)

In [8]:
s=subprocess.Popen(['mkdir','-p',work_dir],
                 stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()

In [20]:
def preamble(MHc=750,MH0=110):
    return '''import model InertDoublet_UFO
define p = g u c d s b u~ c~ d~ s~ b~
define j = p  
define l+ = e+ mu+ 
define l- = e- mu- 
define vl = ve vm vt 
define vl~ = ve~ vm~ vt~

generate p p > h2 h2 j j @0

output ../studies/IDM/BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lambdaL

'''

def lamL_loop(LHA_input_file,MHc=750,MH0=110,lamL=0.01):
    return '''launch ../studies/IDM/BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lambdaL
0    
../'''+LHA_input_file+'''
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL '''+str(lamL)+'''
set mmh0 '''+str(MH0)+'''
set mma0 '''+str(MHc)+'''
set mmhch '''+str(MHc)+'''
0

''' 


def closing(MHc=750,MH0=110):
    return '''launch ../studies/IDM/BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lambdaL -i
print_results --path=./result_BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lamdaL_mh0_110.txt --format=short


done
'''

In [23]:
f=open(work_dir+'/'+work_script,'w')
f.write( preamble(MHc,MH0) )
for lamL in LambdasL:
    f.write( lamL_loop(LHA_input_file,MHc,MH0,lamL=lamL) )
f.write( closing(MHc,MH0) )
f.close()

```bash
# pip3 install subprocess.run && #pip2 install subprocess.run 
```

In [19]:
f=open(MADGRAPH+'/kk.sh','w')
f.write('source '+thisroot+'\n')
f.write('./bin/mg5_aMC ../'+work_dir+'/'+work_script+'\n')
f.close()

In [151]:
s=subprocess.Popen( 'bash kk.sh'.split(), cwd=MADGRAPH,
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [ ]:
s.wait()
(MGOUT,MGERR)=s.communicate()

%%bash
cd madgraph/
./bin/mg5_aMC ../studies/IDM/Task_Asana/BP_750_A_110.txt
cd ..

### Prepare madevent script

In [26]:
pythia_script='TemplateRunPythiaDelphes_all.dat'
if len(LambdasL)>99:
    sys.exit('ERROR: UPDATE FORMAT FOR > 99 runs')
f=open(work_dir+'/'+pythia_script,'w')
for r in range(1,len(LambdasL)+1):
    f.write('pythia run_%02d\n' %r)
    f.write('3\n')
    f.write('0\n')
f.close()

In [27]:
output_dir='studies/IDM/BP_'+str(int(MHc))+'_'+str(int(MH0))+'_vs_lambdaL'

In [28]:
f=open(output_dir+'/kk.sh','w')
f.write('source '+thisroot+'\n')
f.write('./bin/madevent ../'+run_dir+'/'+pythia_script+'\n')
f.close()

In [18]:
s=subprocess.Popen('bash kk.sh'.split(), cwd=output_dir,
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [19]:
s.wait()
(PHOUT,PHERR)=s.communicate()

print(PHOUT.decode('utf-8'))

### Get results

In [ ]:
full_output='output'
s=subprocess.Popen(['mkdir', '-p',full_output]).wait()

#if 'xs_'+str(int(MH0)) not in df.columns.values:
if True:
    for r in range(1,len(LambdasL)+1):
        nrun='%02d' %r
        nrun3='%03d' %r
        s=subprocess.Popen(['cp',output_dir+'/Events/run_'+nrun+'/tag_1_delphes_events.root', 
                            full_output+'/delphes_events_'+str(int(MH0))+'_'+nrun3+'_.root'],
                             stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if s.wait()>0:
            sys.exit('Files not found')

In [7]:
ls studies/IDM/BP_750_110_vs_lambdaL/Events/run_07/

events.lhe.gz  run_07_tag_1_banner.txt  unweighted_events.lhe.gz


### Other

In [ ]:
# %load studies/IDM/Task_Asana/TemplateRunPythiaDelphes_all.dat
pythia run_01
3
0
pythia run_02
3
0
pythia run_03
3
0
pythia run_04
3
0
pythia run_05
3
0
pythia run_06
3
0
pythia run_07
3
0
pythia run_08
3
0
pythia run_09
3
0
pythia run_10
3
0
pythia run_11
3
0
pythia run_12
3
0
pythia run_13
3
0
pythia run_14
3
0
pythia run_15
3
0
pythia run_16
3
0
pythia run_17
3
0
pythia run_18
3
0
pythia run_19
3
0
pythia run_20
3
0


In [ ]:
# %load studies/IDM/Task_Asana/TemplateRunPythiaDelphes_all.dat
pythia run_01 
3 
0 
pythia run_02 
3 
0 
pythia run_03 
3 
0 
pythia run_04 
3 
0 
pythia run_05 
3 
0 
pythia run_06 
3 
0 


In [ ]:
# %load studies/IDM/Task_Asana/BP_750_A_110.txt
import model InertDoublet_UFO
define p = g u c d s b u~ c~ d~ s~ b~
define j = p  
define l+ = e+ mu+ 
define l- = e- mu- 
define vl = ve vm vt 
define vl~ = ve~ vm~ vt~

generate p p > h2 h2 j j @0

output ../studies/IDM/BP_750_110_vs_lambdaL

launch ../studies/IDM/BP_750_110_vs_lambdaL
../MadGraph_cards/benchmarks/param_card_BP750_110.dat
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL 0.01

launch ../studies/IDM/BP_750_110_vs_lambdaL
../MadGraph_cards/benchmarks/param_card_BP750_110.dat
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL 0.02

launch ../studies/IDM/BP_750_110_vs_lambdaL
../MadGraph_cards/benchmarks/param_card_BP750_110.dat
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL 0.05

launch ../studies/IDM/BP_750_110_vs_lambdaL
../MadGraph_cards/benchmarks/param_card_BP750_110.dat
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL 0.07

launch ../studies/IDM/BP_750_110_vs_lambdaL
../MadGraph_cards/benchmarks/param_card_BP750_110.dat
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL 0.1

launch ../studies/IDM/BP_750_110_vs_lambdaL
../MadGraph_cards/benchmarks/param_card_BP750_110.dat
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL 0.15

launch ../studies/IDM/BP_750_110_vs_lambdaL
../MadGraph_cards/benchmarks/param_card_BP750_110.dat
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL 0.2

launch ../studies/IDM/BP_750_110_vs_lambdaL
../MadGraph_cards/benchmarks/param_card_BP750_110.dat
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL 0.25

launch ../studies/IDM/BP_750_110_vs_lambdaL
../MadGraph_cards/benchmarks/param_card_BP750_110.dat
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL 0.3

launch ../studies/IDM/BP_750_110_vs_lambdaL
../MadGraph_cards/benchmarks/param_card_BP750_110.dat
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL 0.35

launch ../studies/IDM/BP_750_110_vs_lambdaL
../MadGraph_cards/benchmarks/param_card_BP750_110.dat
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL 0.4

launch ../studies/IDM/BP_750_110_vs_lambdaL
../MadGraph_cards/benchmarks/param_card_BP750_110.dat
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL 0.45

launch ../studies/IDM/BP_750_110_vs_lambdaL
../MadGraph_cards/benchmarks/param_card_BP750_110.dat
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL 0.5

launch ../studies/IDM/BP_750_110_vs_lambdaL
../MadGraph_cards/benchmarks/param_card_BP750_110.dat
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL 0.55

launch ../studies/IDM/BP_750_110_vs_lambdaL
../MadGraph_cards/benchmarks/param_card_BP750_110.dat
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL 0.6

launch ../studies/IDM/BP_750_110_vs_lambdaL
../MadGraph_cards/benchmarks/param_card_BP750_110.dat
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL 1.0

launch ../studies/IDM/BP_750_110_vs_lambdaL
../MadGraph_cards/benchmarks/param_card_BP750_110.dat
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL 3.0

launch ../studies/IDM/BP_750_110_vs_lambdaL
../MadGraph_cards/benchmarks/param_card_BP750_110.dat
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL 5.0

launch ../studies/IDM/BP_750_110_vs_lambdaL
../MadGraph_cards/benchmarks/param_card_BP750_110.dat
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL 7.0

launch ../studies/IDM/BP_750_110_vs_lambdaL
../MadGraph_cards/benchmarks/param_card_BP750_110.dat
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL 10.0

launch ../studies/IDM/BP_750_110_vs_lambdaL -i
print_results --path=./result_BP_750_110_vs_lamdaL_mh0_110.txt --format=short

done


## Final program

In [13]:
%%writefile index.py
import sys
# pip3 install subprocess.run && pip2 install subprocess.run
# compatibily with last python3
from subprocess import run as RUN
def run(*args,**kwargs):
    return RUN(*args,**kwargs).stderr

import numpy as np
import pandas as pd
# pip3 install pyslha && pip3 install pyslha
import pyslha


def preamble(MHc=750,MH0=110):
    return '''import model InertDoublet_UFO
define p = g u c d s b u~ c~ d~ s~ b~
define j = p  
define l+ = e+ mu+ 
define l- = e- mu- 
define vl = ve vm vt 
define vl~ = ve~ vm~ vt~

generate p p > h2 h2 j j @0

output ../studies/IDM/BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lambdaL

'''

def lamL_loop(MHc=750,MH0=110,lamL=0.01):
    return '''launch ../studies/IDM/BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lambdaL
../MadGraph_cards/benchmarks/param_card_BP'''+str(int(MHc))+'_'+str(int(MH0))+'''.dat
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL '''+str(lamL)+'''

''' 

def closing(MHc=750,MH0=110):
    return '''launch ../studies/IDM/BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lambdaL -i
print_results --path=./result_BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lamdaL_mh0_110.txt --format=short

done
'''

MHc=750
MH0=260
MADGRAPH='madgraph'

LambdasL=[0.01,0.02,0.05,0.07,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,1.0,3.0,5.0,7.0,10.0]

#prepare dirs
run('mkdir -p studies/IDM/Task_Asana')


clone=False
if clone:
    run('git clone  --recursive git@gitlab.com:restrepo/VBF_IDM.git')
    f=open('kk.sh','w')
    f.write('cd '+MADGRAPH+'/\n')
    f.write('git checkout -b v2.3.3\n')
    f.write('./bin/mg5_aMC install.dat > /tmp/kk 2> /tmp/kkk')
    f.close()
    run('bash kk.sh')
    
#calculates cross section    
cs_file='BP_'+str(int(MHc))+'_A_'+str(int(MH0))+'.txt'    
f=open('studies/IDM/Task_Asana/'+cs_file,'w')
f.write( preamble(MHc,MH0) )
for lamL in LambdasL:
    f.write( lamL_loop(MHc,MH0,lamL=lamL) )
f.write( closing(MHc,MH0) )
f.close()

f=open('kk.sh','w')
f.write('cd '+MADGRAPH+'\n./bin/mg5_aMC ../studies/IDM/Task_Asana/'+cs_file+' > /tmp/kk' )
f.close()

run('bash kk.sh')

# run pythia
if len(LambdasL)>99:
    sys.exit('ERROR: UPDATE FORMAT FOR > 99 runs')
f=open('studies/IDM/Task_Asana/TemplateRunPythiaDelphes_all.dat','w')
for r in range(1,len(LambdasL)+1):
    f.write('pythia run_%02d\n' %r)
    f.write('3\n')
    f.write('0\n')
f.close()

f=open('kk.sh','w')
f.write('cd studies/IDM/BP_'+str(int(MHc))+'_'+str(int(MH0))+'_vs_lambdaL\n')
f.write('./bin/madevent ../Task_Asana/TemplateRunPythiaDelphes_all.dat')
f.close()

run('bash kk.sh')

f=open('kk.sh','w')
f.write('mkdir -p output\n')

#if 'xs_'+str(int(MH0)) not in df.columns.values:
if True:
    for r in range(1,len(LambdasL)+1):
        nrun='%02d' %r
        nrun3='%03d' %r
        f.write('cp studies/IDM/BP_'+str(int(MHc))+'_'+str(int(MH0))+ 
              '_vs_lambdaL/Events/run_'+nrun+'/tag_1_delphes_events.root output/delphes_events_'+str(int(MH0))+
              '_'+nrun3+'_.root\n')
    
f.close()

run('bash kk.sh')

Writing index.py


## Trash area

Temporal area where preliminary approaches are tried for.